In [ ]:
!pip install gradio

In [4]:
import gradio as gr
import pandas as pd
import datetime
import pandas as pd
from bokeh.models import ColumnDataSource, FactorRange, Range1d
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Blues3
from bokeh.models import FixedTicker
from bokeh.models import HoverTool
from bokeh.models import LabelSet
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
def process_RoomList(file1):
    df = pd.read_excel(file1.name)
    df = df[df.Campus == "TLB"]
    df = df[df.Type == "Classroom (110)"]
    df = df[df.Description !="Restricted"]
    df = df[~df.Name.str.contains("Expired")]
    df['RoomID'] = df['Building Code'] + " " + df['Number'].astype(str)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    RoomList_processed = df.drop(columns=['Building Code', 'Number'])

    return RoomList_processed

def process_Sections(file2):
    df2 = pd.read_excel(file2.name)
    df2 = df2[df2.Campus == "TLB"]
    df2['Room'] = df2['Room'].str.split('-').str[0].str.strip()
    df2.rename(columns={'Room':'RoomID'}, inplace=True)
    cols = df2.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df2 = df2[cols]

    Sections_processed = df2

    return Sections_processed

def process_RoomList_Sections(file1, file2):
    df = pd.read_excel(file1.name)
    df = df[df.Campus == "TLB"]
    df = df[df.Type == "Classroom (110)"]
    df = df[df.Description !="Restricted"]
    df = df[~df.Name.str.contains("Expired")]
    df['RoomID'] = df['Building Code'] + " " + df['Number'].astype(str)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    RoomList_processed = df.drop(columns=['Building Code', 'Number'])

    df2 = pd.read_excel(file2.name)
    df2 = df2[df2.Campus == "TLB"]
    df2['Room'] = df2['Room'].str.split('-').str[0].str.strip()
    df2.rename(columns={'Room':'RoomID'}, inplace=True)
    cols = df2.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df2 = df2[cols]

    Sections_processed = df2

    return RoomList_processed, Sections_processed



def show_roominfo_schedule(roomid, startdate, enddate, RoomList_file, Sections_file):
    # Convert string to datetime
    startdate = datetime.datetime.strptime(startdate, "%m/%d/%Y").date()
    enddate = datetime.datetime.strptime(enddate, "%m/%d/%Y").date()

    # Process the files
    RoomList_processed, Sections_processed = process_RoomList_Sections(RoomList_file, Sections_file)

    # Input roomID, return this room information
    room_info= RoomList_processed[(RoomList_processed['RoomID'] == roomid)]

    # Input roomID, date range, return this room schedule
    Sections_processed_copy = Sections_processed[(Sections_processed['RoomID'] == roomid)].copy()
    Sections_processed_copy['Start Date'] = pd.to_datetime(Sections_processed_copy['Start Date']).dt.date
    Sections_processed_copy['End Date'] = pd.to_datetime(Sections_processed_copy['End Date']).dt.date
    room_schedule = Sections_processed_copy[(Sections_processed_copy['Start Date'] <= enddate) & (Sections_processed_copy['End Date'] >= startdate)]

    room_info_str = room_info.to_html()
    room_schedule_str = room_schedule.to_html()

    return room_info_str, room_schedule_str

#visualize schedule

def room_schedule_visualization(search_room,search_date_start_str,search_date_end_str, Sections_file):

    Sections_processed = process_Sections(Sections_file)
    df = Sections_processed
    search_date_start = datetime.datetime.strptime(search_date_start_str, '%m/%d/%Y').date()
    search_date_end = datetime.datetime.strptime(search_date_end_str, '%m/%d/%Y').date()

    df_copy = df[(df['RoomID'] == search_room)].copy()
    df_copy['Start Date'] = pd.to_datetime(df_copy['Start Date']).dt.date
    df_copy['End Date'] = pd.to_datetime(df_copy['End Date']).dt.date

    search_date_start = datetime.datetime.strptime(search_date_start_str, '%m/%d/%Y').date()
    search_date_end = datetime.datetime.strptime(search_date_end_str, '%m/%d/%Y').date()
    df_rooms = df_copy[(df_copy['Start Date'] <= search_date_end) & (df_copy['End Date'] >= search_date_start)]
    room_schedule = df_rooms[df_rooms['Start Date'] != df_rooms['End Date']]
    #filter same day schedule
    room_schedule_list = []
    for index, row in room_schedule.iterrows():
        room_schedule_list.append([row['Course/Section'], row['Days Met'],row['Start Time'], row['End Time'],row['Start Date'],row['End Date']])

    room_schedule_list2 = []
    for i in room_schedule_list:
        start_time1 = datetime.datetime.strptime(i[2], '%I:%M %p').strftime('%I:%M %p')
        end_time1 = datetime.datetime.strptime(i[3], '%I:%M %p').strftime('%I:%M %p')
        start_time = datetime.datetime.strptime(i[2], '%I:%M %p').time()
        end_time = datetime.datetime.strptime(i[3], '%I:%M %p').time()
        start_minutes = start_time.hour * 60 + start_time.minute
        end_minutes = end_time.hour * 60 + end_time.minute
        start_date = i[4].strftime('%m/%d/%Y')  # Convert date to string
        end_date = i[5].strftime('%m/%d/%Y')    # Convert date to string
        room_schedule_list2.append([i[0], i[1], start_minutes, end_minutes,start_time1,end_time1,start_date,end_date])

    split_schedules = split_timeslots(room_schedule_list2)

    df = pd.DataFrame(split_schedules, columns=['Course', 'Day', 'Start', 'End','Timeslot','Start Date','End Date'])
    df['Middle'] = (df['Start'] + df['End']) / 2
    source = ColumnDataSource(df)

    p = figure(x_range=FactorRange(*'MTWRF'), y_range=Range1d(1320, 360),
               x_axis_location="above", y_axis_location="left",
               width=800, height=600, toolbar_location=None,title=search_room,title_location = 'below')
    hover = HoverTool(tooltips=[
        ("Course", "@Course"),
        ("Timeslot", "@Timeslot"),
        ("Start Date", "@{Start Date}"),
        ("End Date", "@{End Date}")])

    p.add_tools(hover)

    p.vbar(x='Day', bottom='Start', top='End', width=0.8, color=Blues3[1], source=source)

    labels = LabelSet(x='Day', y='Middle', text='Course', level='glyph',
                  x_offset=-60, y_offset=0, source=source, text_color='black',text_font_size='8pt')
    p.add_layout(labels)

    labels2 = LabelSet(x='Day', y='Middle', text='Timeslot', level='glyph',
                  x_offset=-60, y_offset=-15, source=source, text_color='black',text_font_size='8pt')
    p.add_layout(labels2)

    times = ['06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00',
                     '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00',
                     '20:00', '21:00', '22:00']
    time_ticks = list(range(360, 1321, 60))

    p.yaxis.ticker = FixedTicker(ticks=time_ticks)

    p.yaxis.major_label_overrides = dict(zip(time_ticks, times))

    output_notebook()
    # show(p)
    return p

def split_timeslots(schedules_list):
    split_schedules = []
    for schedule in schedules_list:
        course, days, start_time, end_time,start_time1,end_time1,start_date,end_date = schedule
        start_time = start_time
        end_time = end_time
        start_time1 = start_time1
        end_time1 = end_time1
        start_date = start_date
        end_date = end_date
        for day in days:
            time_slot = f"{day} {start_time1}-{end_time1}"
            split_schedules.append([course, day, start_time, end_time,time_slot,start_date,end_date])

    return split_schedules

def show_rooms_with_capacity_raiderroom(class_size, raider_room, RoomList_file):
    RoomList_processed = process_RoomList(RoomList_file)
    df2 = RoomList_processed
    df2 = df2[(df2['Capacity'] >= class_size)]
    if raider_room:
        df2 = df2[(df2['Description'] == 'Raider Room')]
    return df2.to_html()



def find_rooms(class_size, raider_room, search_date_start_str, search_date_end_str, search_time_start_str, search_time_end_str, search_day_str,RoomList_processed,Sections_processed):
    RoomList_processed, Sections_processed = process_RoomList_Sections(RoomList_processed, Sections_processed)

    search_date_start = datetime.datetime.strptime(search_date_start_str, '%m/%d/%Y').date()
    search_date_end = datetime.datetime.strptime(search_date_end_str, '%m/%d/%Y').date()

    search_time_start = datetime.datetime.strptime(search_time_start_str, '%I:%M %p').time()
    search_time_end = datetime.datetime.strptime(search_time_end_str, '%I:%M %p').time()

    search_day = list(search_day_str)

    df2 = RoomList_processed.copy()
    df3 = Sections_processed.copy()

    df2 = df2[(df2['Capacity'] >= class_size)]
    if raider_room:
        df2 = df2[(df2['Description'] == 'Raider Room')]

    roomID_list = df2['RoomID'].tolist()

    df3['Start Date'] = pd.to_datetime(df3['Start Date']).dt.date
    df3['End Date'] = pd.to_datetime(df3['End Date']).dt.date

    if df3['Start Time'].dtype == object:
        df3['Start Time'] = pd.to_datetime(df3['Start Time'], format='%I:%M %p').dt.time
    if df3['End Time'].dtype == object:
        df3['End Time'] = pd.to_datetime(df3['End Time'], format='%I:%M %p').dt.time

    df_rooms = df3[(df3['Start Date'] <= search_date_end) & (df3['End Date'] >= search_date_start)]
    room_schedules = df_rooms[df_rooms['Start Date'] != df_rooms['End Date']]
    #filter same day schedule

    room_schedule_lists = [room_schedules[room_schedules['RoomID'] == roomID] for roomID in roomID_list]

    room_schedule_lists2 = [room_schedule[room_schedule['Days Met'].str.contains('|'.join(search_day))] for room_schedule in room_schedule_lists]

    new_roomID_list = []
    for roomID in roomID_list:
        is_available = True
        found_schedule = False
        for room_schedule in room_schedule_lists2:
            if not room_schedule.empty and room_schedule['RoomID'].iloc[0] == roomID:
                found_schedule = True
                for index, schedule_row in room_schedule.iterrows():
                    start_time = schedule_row['Start Time']
                    end_time = schedule_row['End Time']
                    if start_time <= search_time_end and end_time >= search_time_start:
                        is_available = False
                        break
                if not is_available:
                    break
        if found_schedule and is_available:
            new_roomID_list.append(roomID)
    formatted_result = "\n".join(new_roomID_list)
    return formatted_result
    # return new_roomID_list


def find_rooms_with_time_slots(class_time_slots, capacity, raiderroom, startdate, enddate,RoomList_processed,Sections_processed):
    RoomList_processed, Sections_processed = process_RoomList_Sections(RoomList_processed, Sections_processed)
    room_availabilities = {}
    result_str = ""
    for class_time in class_time_slots:
        # Split the string and get day, start time and end time
        day_time, time_range = class_time.split()
        time_start_str, time_end_str = time_range.split('-')

        # Convert time in 24-hour format to 12-hour format
        time_start_str = datetime.datetime.strptime(time_start_str, "%H%M").strftime("%I:%M %p")
        time_end_str = datetime.datetime.strptime(time_end_str, "%H%M").strftime("%I:%M %p")

        search_date_start = datetime.datetime.strptime(startdate, '%m/%d/%Y').date()
        search_date_end = datetime.datetime.strptime(enddate, '%m/%d/%Y').date()

        search_time_start = datetime.datetime.strptime(time_start_str, '%I:%M %p').time()
        search_time_end = datetime.datetime.strptime(time_end_str, '%I:%M %p').time()

        search_day = list(day_time)

        df2 = RoomList_processed.copy()
        df3 = Sections_processed.copy()

        df2 = df2[(df2['Capacity'] >= capacity)]
        if raiderroom:
            df2 = df2[(df2['Description'] == 'Raider Room')]

        roomID_list = df2['RoomID'].tolist()

        df3['Start Date'] = pd.to_datetime(df3['Start Date']).dt.date
        df3['End Date'] = pd.to_datetime(df3['End Date']).dt.date

        if df3['Start Time'].dtype == object:
            df3['Start Time'] = pd.to_datetime(df3['Start Time'], format='%I:%M %p').dt.time
        if df3['End Time'].dtype == object:
            df3['End Time'] = pd.to_datetime(df3['End Time'], format='%I:%M %p').dt.time

        df_rooms = df3[(df3['Start Date'] <= search_date_end) & (df3['End Date'] >= search_date_start)]
        room_schedules = df_rooms[df_rooms['Start Date'] != df_rooms['End Date']]
        #filter same day schedule

        room_schedule_lists = [room_schedules[room_schedules['RoomID'] == roomID] for roomID in roomID_list]

        room_schedule_lists2 = [room_schedule[room_schedule['Days Met'].str.contains('|'.join(search_day))] for room_schedule in room_schedule_lists]

        new_roomID_list = []
        for roomID in roomID_list:
            is_available = True
            found_schedule = False
            for room_schedule in room_schedule_lists2:
                if not room_schedule.empty and room_schedule['RoomID'].iloc[0] == roomID:
                    found_schedule = True
                    for index, schedule_row in room_schedule.iterrows():
                        start_time = schedule_row['Start Time']
                        end_time = schedule_row['End Time']
                        if start_time <= search_time_end and end_time >= search_time_start:
                            is_available = False
                            break
                    if not is_available:
                        break
            if found_schedule and is_available:
                new_roomID_list.append(roomID)

        # Add the list of available rooms to the dictionary
        room_availabilities[class_time] = new_roomID_list
        if len(new_roomID_list) != 0:
            result_str += f"For the time slot {class_time}, the following rooms are available: {new_roomID_list}\n-----------------------------------------\n"

    # return the dictionary containing room availability for each time slot
    return result_str


#Export classroom property and availability.xlsx

def find_rooms_2(class_size, raider_room, search_date_start_str, search_date_end_str, search_time_start_str, search_time_end_str, search_day_str,RoomList_processed,Sections_processed):

    search_date_start = datetime.datetime.strptime(search_date_start_str, '%m/%d/%Y').date()
    search_date_end = datetime.datetime.strptime(search_date_end_str, '%m/%d/%Y').date()

    search_time_start = datetime.datetime.strptime(search_time_start_str, '%I:%M %p').time()
    search_time_end = datetime.datetime.strptime(search_time_end_str, '%I:%M %p').time()

    search_day = list(search_day_str)

    df2 = RoomList_processed.copy()
    df3 = Sections_processed.copy()

    df2 = df2[(df2['Capacity'] >= class_size)]
    if raider_room:
        df2 = df2[(df2['Description'] == 'Raider Room')]

    roomID_list = df2['RoomID'].tolist()

    df3['Start Date'] = pd.to_datetime(df3['Start Date']).dt.date
    df3['End Date'] = pd.to_datetime(df3['End Date']).dt.date

    if df3['Start Time'].dtype == object:
        df3['Start Time'] = pd.to_datetime(df3['Start Time'], format='%I:%M %p').dt.time
    if df3['End Time'].dtype == object:
        df3['End Time'] = pd.to_datetime(df3['End Time'], format='%I:%M %p').dt.time

    df_rooms = df3[(df3['Start Date'] <= search_date_end) & (df3['End Date'] >= search_date_start)]
    room_schedules = df_rooms[df_rooms['Start Date'] != df_rooms['End Date']]
#filter same day schedule
    room_schedule_lists = [room_schedules[room_schedules['RoomID'] == roomID] for roomID in roomID_list]

    room_schedule_lists2 = [room_schedule[room_schedule['Days Met'].str.contains('|'.join(search_day))] for room_schedule in room_schedule_lists]

    new_roomID_list = []
    for roomID in roomID_list:
        is_available = True
        found_schedule = False
        for room_schedule in room_schedule_lists2:
            if not room_schedule.empty and room_schedule['RoomID'].iloc[0] == roomID:
                found_schedule = True
                for index, schedule_row in room_schedule.iterrows():
                    start_time = schedule_row['Start Time']
                    end_time = schedule_row['End Time']
                    if start_time <= search_time_end and end_time >= search_time_start:
                        is_available = False
                        break
                if not is_available:
                    break
        if found_schedule and is_available:
            new_roomID_list.append(roomID)

    return new_roomID_list

def find_rooms_with_time_slots_2(class_time_slots, capacity, raiderroom, startdate, enddate, RoomList_processed, Sections_processed):
    room_availabilities = {}
    for class_time in class_time_slots:
        # Split the string and get day, start time and end time
        day_time, time_range = class_time.split()
        time_start_str, time_end_str = time_range.split('-')

        # Convert time in 24-hour format to 12-hour format
        time_start_str = datetime.datetime.strptime(time_start_str, "%H%M").strftime("%I:%M %p")
        time_end_str = datetime.datetime.strptime(time_end_str, "%H%M").strftime("%I:%M %p")

        # Get the list of available rooms
        available_rooms = find_rooms_2(capacity, raiderroom, startdate, enddate,
                                     time_start_str, time_end_str,
                                     day_time, RoomList_processed, Sections_processed)

        # Add the list of available rooms to the dictionary
        room_availabilities[class_time] = available_rooms

    # return the dictionary containing room availability for each time slot
    return room_availabilities
def export(startdate,enddate, RoomList_file, Sections_file):

    output_file='/content/classroom property and availability.xlsx'
    output_file2='/content/viewer version_classroom property and availability.xlsx'

    class_time_slots=['MWF 0800-0850', 'MWF 0900-0950','MWF 1000-1050','MWF 1100-1150', 'MWF 1200-1250', 'MWF 1300-1350', 'MWF 1400-1450', 'MWF 1500-1550','MWF 1600-1650','MWF 1700-1750',
                  'TR 0800-0920','TR 0930-1050', 'TR 1100-1220', 'TR 1230-1350', 'TR 1400-1520', 'TR 1700-1820', 'TR 1830-1950', 'TR 2000-2120',
                  'M 1500-1750','M 1600-1850','M 1630-2120','T 1630-2120','W 1500-1750','W 1600-1850','W 1630-2120','R 1630-2120','F 1500-1750','F 1600-1850','F 1630-2120']

    building_name_list=['English & Philosophy',
    'Lanier Center',
    'Engineering Computer Center',
    'Chemical Engineering',
    'Mechanical Engineering South',
    'Media & Communication',
    'Animal & Food Sciences',
    'Art',
    'Electrical Engineering',
    'Psychology',
    'Industrial, Manufacturing and Systems Engineering',
    'Architecture',
    'Education',
    'Rawls College of Business Administration',
    'Creative Movement Studio',
    'Human Sciences',
    'Science',
    'Classical & Modern Languages & Literatures',
    'Livermore Center',
    'Biology',
    'Mechanical Engineering North',
    'Chemistry',
    'Mathematical Sciences',
    'Agriculture Sciences',
    'Goddard Range & Wildlife Management',
    'Terry Fuller Petroleum Engineering Research Building',
    'Holden Hall',
    'Civil Engineering',
    'Kinesiology and Sport Management',
    'Law Building',
    'Agricultural Ed & Communications',
    'Bayer Plant Science (South Building)',
    'Livestock Arena & Meat Lab']

    distances=[0.6,
 1.1,
 0.0,
 0.2,
 0.1,
 0.6,
 0.3,
 0.7,
 0.1,
 0.6,
 0.2,
 0.8,
 0.7,
 0.4,
 0.2,
 0.4,
 0.3,
 0.7,
 0.0,
 0.4,
 0.1,
 0.4,
 0.3,
 0.4,
 0.5,
 0.4,
 0.2,
 0.2,
 0.8,
 1.1,
 0.5,
 0.6,
 1.1]
    building_distance_dict = dict(zip(building_name_list, distances))

    Sections_processed = process_Sections(Sections_file)

    df = pd.read_excel(RoomList_file.name)
    df = df[df.Campus == "TLB"]
    df = df[df.Type == "Classroom (110)"]
    df = df[df.Description !="Restricted"]
    df = df[~df.Name.str.contains("Expired")]
    df['RoomID'] = df['Building Code'] + " " + df['Number'].astype(str)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    RoomList_processed = df.drop(columns=['Building Code', 'Number'])
    RoomList_processed = RoomList_processed.drop(columns=['Name'])
    RoomList_processed['Distance'] = RoomList_processed['Building Name'].map(building_distance_dict)

    with pd.ExcelWriter(output_file) as writer:
        RoomList_processed.to_excel(writer, sheet_name='property', index=False)
        RoomList_processed2 = RoomList_processed.drop(columns=['Capacity','Description','Distance','Type','Campus','Room','Building Name'])
        RoomList_processed2.to_excel(writer, sheet_name='availability', index=False)

    room_availabilities = find_rooms_with_time_slots_2(class_time_slots, 0, False, startdate, enddate, RoomList_processed, Sections_processed)

    df = pd.read_excel(output_file)
    wb = openpyxl.load_workbook(output_file)
    ws = wb['availability']
    for i in range(len(class_time_slots)):
        ws.cell(row=1, column=i+2).value = class_time_slots[i]

    for i in range(len(class_time_slots)):
        for j in range(len(df)):
            if df.iloc[j, 0] in room_availabilities[class_time_slots[i]]:
                ws.cell(row=j+2, column=i+2).value = 1
            else:
                ws.cell(row=j+2, column=i+2).value = 0

    wb.save(output_file)

    # Load the data from the two sheets into pandas dataframes
    df1 = pd.read_excel(output_file, sheet_name=0)
    df2 = pd.read_excel(output_file, sheet_name=1)

    # Merge the two dataframes on RoomID
    merged_df = pd.merge(df1, df2, on="RoomID", how="outer")

    with pd.ExcelWriter(output_file2) as writer:
        merged_df.to_excel(writer, index=False)

    # Define a fill pattern for cells
    green_fill = PatternFill(start_color="00FF00",
                             end_color="00FF00",
                             fill_type="solid")

    # Load the workbook
    book = load_workbook(output_file2)

    # Access the sheet you want to modify
    sheet = book.active

    # Apply the fill pattern to cells with value 1
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value == 1:
                cell.fill = green_fill

    # Save the changes
    book.save(output_file2)

    return output_file, output_file2


with gr.Blocks() as demo:
    with gr.Tab("Room info and schedule search"):
        files_input1 = gr.File(type="filepath", label="RoomList File")
        files_input2 = gr.File(type="filepath", label="Sections File")
        roomid = gr.Textbox(label="Room ID",value="ELECE 00101")




        startdate = gr.Textbox(label="Start Date (MM/DD/YYYY)",value="8/24/2023")
        enddate = gr.Textbox(label="End Date (MM/DD/YYYY)",value="12/24/2023")

        # button1 = gr.Button("Preprocess")
        button1 = gr.Button("Search")
        textbox_output1 = gr.HTML()
        textbox_output2 = gr.HTML()

        button2 = gr.Button("Visualization")
        # gr.Markdown("See more details in the colab terminal if overlap")
        img_outputs = gr.Plot()


    with gr.Tab("Available rooms search with classsize"):
        classsize = gr.Number(label="Class Size",value=50)
        raiderroom = gr.Checkbox(label="Raider Room")
        button3 = gr.Button("Search")
        textbox_output3 = gr.HTML()

    button1.click(show_roominfo_schedule,
                  inputs=[roomid, startdate, enddate, files_input1, files_input2],
                  outputs=[textbox_output1, textbox_output2])
    button2.click(room_schedule_visualization,
                  inputs=[roomid,startdate,enddate, files_input2],
                  outputs=img_outputs)
    button3.click(show_rooms_with_capacity_raiderroom,
                      inputs=[classsize, raiderroom, files_input1],
                      outputs=textbox_output3)
    with gr.Tab("Available rooms search with time range"):
        classsize2 = gr.Number(label="Class Size",value=50)
        raiderroom2 = gr.Checkbox(label="Raider Room")
        startdate2 = gr.Textbox(label="Start Date (MM/DD/YYYY)",value="8/24/2023")
        enddate2 = gr.Textbox(label="End Date (MM/DD/YYYY)",value="12/24/2023")

        class_time_slots=['MWF 0800-0850', 'MWF 0900-0950','MWF 1000-1050','MWF 1100-1150', 'MWF 1200-1250', 'MWF 1300-1350', 'MWF 1400-1450', 'MWF 1500-1550','MWF 1600-1650','MWF 1700-1750',
                          'TR 0800-0920','TR 0930-1050', 'TR 1100-1220', 'TR 1230-1350', 'TR 1400-1520', 'TR 1700-1820', 'TR 1830-1950', 'TR 2000-2120']
        class_time_checkbox = gr.CheckboxGroup(label="Class Times", choices=class_time_slots)
        button5 = gr.Button("Search")
        textbox_output4 = gr.Textbox(lines=6)

        starttime2 = gr.Textbox(label="Start Time (HH:MM P)",value="04:30 PM")
        endtime2 = gr.Textbox(label="End Time (HH:MM P)",value="09:20 PM")
        weekdays = gr.Textbox(label="Week Days (MTWRF)",value="M")
        button4 = gr.Button("Search")
        textbox_output5 = gr.Textbox(lines=6)


    button4.click(find_rooms,
                  inputs=[classsize2, raiderroom2, startdate2, enddate2, starttime2, endtime2, weekdays, files_input1, files_input2],
                  outputs=textbox_output5)

    button5.click(find_rooms_with_time_slots,
                inputs=[class_time_checkbox, classsize2, raiderroom2, startdate2, enddate2, files_input1, files_input2],
                outputs=textbox_output4)

    with gr.Tab("Export classroom property and availability.xlsx"):
      startdate3 = gr.Textbox(label="Start Date (MM/DD/YYYY)",value="8/24/2023")
      enddate3 = gr.Textbox(label="End Date (MM/DD/YYYY)",value="12/24/2023")
      file_output1 = gr.File(label="classroom property and availability.xlsx")
      file_output2 = gr.File(label="viewer version_classroom property and availability.xlsx")
      button6 = gr.Button("Export")

    button6.click(export,
                  inputs=[startdate3,enddate3,files_input1, files_input2],
                  outputs=[file_output1,file_output2])

demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://695be648b3b8f11965.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://695be648b3b8f11965.gradio.live
